<a href="https://colab.research.google.com/github/juasalazarmo/Se-ales_Sistemas/blob/main/Desarrollo_Parcial_1_SyS_2024_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.

## Estudiante: Juan Carlos Salazar Moreno
### CC 1060586070


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

# **Desarrollo pregunta 1**

**Primero definimos la señal continua $x(t)$ y luego la discretizamos**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Definimos la señal continua x(t)
def x_t(t):
    return 0.3 * np.cos(1000 * np.pi * t - np.pi / 4) + 0.6 * np.sin(2000 * np.pi * t) + 0.1 * np.cos(11000 * np.pi * t - np.pi)

# Parámetros de simulación
Fs = 5000  # Frecuencia de muestreo en Hz
Ts = 1 / Fs  # Período de muestreo
duration = 3 / 1000  # Duración para al menos 3 ciclos (ciclo más largo es 1ms)
t_cont = np.linspace(0, duration, 10000)  # Tiempo continuo
t_disc = np.arange(0, duration, Ts)  # Tiempo discreto (muestreo)

# Señal continua y muestreada
x_cont = x_t(t_cont)
x_disc = x_t(t_disc)

# Graficar las señales continua y muestreada
plt.figure(figsize=(10, 6))
plt.plot(t_cont, x_cont, label="Señal continua x(t)")
plt.stem(t_disc, x_disc, linefmt='r-', markerfmt='ro', basefmt=' ', label="Señal muestreada")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.title("Señal continua y su muestreo")
plt.legend()
plt.grid()
plt.show()


**Ahora cuantizamos la señal mustreada anteriormente**

In [ ]:
# Cuantización con 5 bits
bits = 5
levels = 2**bits  # Número de niveles de cuantización
V_min, V_max = -3.3, 3.3  # Rango del ADC
step_size = (V_max - V_min) / levels  # Tamaño del paso de cuantización

# Función de cuantización
def quantize(signal, V_min, V_max, step_size):
    quantized_signal = np.round((signal - V_min) / step_size) * step_size + V_min
    quantized_signal = np.clip(quantized_signal, V_min, V_max)  # Limitar al rango del ADC
    return quantized_signal

# Señal cuantizada
x_quant = quantize(x_disc, V_min, V_max, step_size)

# Graficar la señal cuantizada
plt.figure(figsize=(10, 6))
plt.stem(t_disc, x_disc, linefmt='b-', markerfmt='bo', basefmt=' ', label="Señal muestreada")
plt.stem(t_disc, x_quant, linefmt='r-', markerfmt='ro', basefmt=' ', label="Señal cuantizada")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.title("Señal muestreada y cuantizada")
plt.legend()
plt.grid()
plt.show()


**Como tercer paso debemos verificar is el muestreo es adecuado por medio del Teorrema de Nyquist. Si la frecuencia de la señal más alta $(11 kHz)$ es mayor que $F_s /2 = 2.5kHz$, el muestreo srá inapropiado.**

In [ ]:
# Frecuencia máxima en la señal continua
f_max = 11000 / 2  # Frecuencia de la señal más alta

if Fs / 2 < f_max:
    print(f"La frecuencia de muestreo Fs={Fs} Hz NO es adecuada. F_max={f_max} Hz excede Fs/2={Fs/2} Hz.")
else:
    print(f"La frecuencia de muestreo Fs={Fs} Hz es adecuada.")


**Como el muestreo no cumple Nyquist, diseñaremos un conversor análogo digital configurable**

In [ ]:
# ADC configurable
class ADC:
    def __init__(self, bits=5, V_min=-3.3, V_max=3.3, Fs=5000):
        self.bits = bits
        self.V_min = V_min
        self.V_max = V_max
        self.Fs = Fs
        self.step_size = (V_max - V_min) / (2**bits)

    def quantize(self, signal):
        """Cuantiza la señal."""
        quantized_signal = np.round((signal - self.V_min) / self.step_size) * self.step_size + self.V_min
        return np.clip(quantized_signal, self.V_min, self.V_max)

    def is_sampling_adequate(self, f_max):
        """Verifica el teorema de Nyquist."""
        return self.Fs / 2 >= f_max

    def process_signal(self, signal_func, duration, f_max):
        """Procesa la señal: continuo, muestreo y digitalización."""
        # Tiempo continuo y discreto
        t_cont = np.linspace(0, duration, 10000)
        t_disc = np.arange(0, duration, 1 / self.Fs)

        # Señales continua y muestreada
        x_cont = signal_func(t_cont)
        x_disc = signal_func(t_disc)

        # Señal cuantizada
        x_quant = self.quantize(x_disc)

        # Verificación del muestreo
        adequate = self.is_sampling_adequate(f_max)

        return t_cont, x_cont, t_disc, x_disc, x_quant, adequate

# Configuración inicial
bits = 5
V_min, V_max = -3.3, 3.3
Fs = 5000
f_max = 11000 / 2  # Frecuencia máxima de la señal

# Crear instancia del ADC
adc = ADC(bits=bits, V_min=V_min, V_max=V_max, Fs=Fs)

# Duración: al menos 3 ciclos de la señal de menor frecuencia
duration = 3 / 1000  # Ciclo más largo es 1 ms

# Procesar la señal
t_cont, x_cont, t_disc, x_disc, x_quant, adequate = adc.process_signal(x_t, duration, f_max)

# Graficar resultados
plt.figure(figsize=(12, 8))

# Señal continua
plt.subplot(3, 1, 1)
plt.plot(t_cont, x_cont, label="Señal continua x(t)")
plt.title("Señal continua")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()

# Señal muestreada
plt.subplot(3, 1, 2)
plt.stem(t_disc, x_disc, linefmt='g-', markerfmt='go', basefmt=' ', label="Señal muestreada")
plt.title("Señal muestreada")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()

# Señal digitalizada
plt.subplot(3, 1, 3)
plt.stem(t_disc, x_quant, linefmt='r-', markerfmt='ro', basefmt=' ', label="Señal digitalizada")
plt.title("Señal digitalizada")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

# Resultado de la evaluación
if adequate:
    print(f"La frecuencia de muestreo Fs={Fs} Hz es adecuada según el teorema de Nyquist.")
else:
    print(f"La frecuencia de muestreo Fs={Fs} Hz NO es adecuada. Se requiere Fs > {2 * f_max} Hz.")


# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

#**Desarrollo pregunta 2**

Para resolver el ejercicio debemos tener en cuenta los siguientes puntos:
1. Analizar si el **sistema es lineal**: Debe cumplir la **aditividad** (la salida de la suma de dos entradas es igual a la suma de las salidas).La función **$np.allclose()$** es adecuada para comprobar si las salidas son prácticamente iguales, considerando errores numéricos.

2. Analizar la **invarianza en el tiempo**: Si se cumple que un desplazamiento en la entrada provoca el mismo desplazamiento en la salida.

3. Si se trata de un SLIT, su respuesta al impulso $h[n]$ puede determinarse. Luego, usando la convolución,por medio de la función **$lfilter()$** podemos encontrar la salida para $x[n]$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_  # Asegúrate de tener el módulo P1_model disponible
from scipy.signal import lfilter

# Cargar el sistema
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs  # Frecuencia de muestreo

# 1. ANÁLISIS DE LINEALIDAD

# Generar dos señales de entrada
n = np.arange(-50, 51)  # Definir el rango de n para las señales
x1 = np.sin( np.pi * n)
x2 = np.sin( np.pi * n)

# Obtener las salidas para x1 y x2
y1 = my_system.predict(x1)
y2 = my_system.predict(x2)
y_sum = my_system.predict(x1 + x2)

# Verificar la propiedad de linealidad
if np.allclose(y1 + y2, y_sum):
    print("El sistema es lineal.")
else:
    print("El sistema NO es lineal.")

# Análisis de invarianza en el tiempo
shift = 5  # Desplazamiento arbitrario
x_shifted = np.sin(100 * np.pi * (n - shift))
y_shifted = my_system.predict(x_shifted)
y_direct = my_system.predict(np.sin(100 * np.pi * n))

if np.allclose(y_shifted, y_direct):
    print("El sistema es invariante en el tiempo.")
else:
    print("El sistema NO es invariante en el tiempo.")

# 2. OBTENER LA RESPUESTA AL IMPULSO
delta = np.zeros_like(n)
delta[len(n)//2] = 1  # Impulso unitario en el centro
h = my_system.predict(delta)

# 3. SIMULAR LA SEÑAL DADA: x[n] = sin(100πn) + sin(600πn)
x = np.sin(100 * np.pi * n) + np.sin(600 * np.pi * n)
y = lfilter(h, 1, x)

# Visualización de las señales
fig, axs = plt.subplots(3, 1, figsize=(10, 8))

axs[0].stem(n, x, label='Señal de entrada x[n]')
axs[0].set_xlabel('n')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()

axs[1].stem(n, h, label='Respuesta al impulso h[n]')
axs[1].set_xlabel('n')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()

axs[2].stem(n, y, label='Señal de salida y[n]')
axs[2].set_xlabel('n')
axs[2].set_ylabel('Amplitud')
axs[2].legend()
axs[2].grid()

plt.tight_layout()
plt.show()


##**AYUDAS DE INTELIGENCIA ARTIFICIAL**

####Se implementó inteligencia artificial (ChatGPT) para aclarar algunas dudas y poder tener una guía al momento de resolver ambas preguntas.


####**Instrucción en ChatGPT para la primera pregunta:** "Desarrollar un código base que simule la digitalización de una señal continua usando un conversor análogo-digital con parámetros configurables como el número de bits, el rango de entrada y la frecuencia de muestreo. Además, debía incluir la verificación de si la frecuencia es adecuada y graficar la señal continua, discreta y digital."

####**Instrucción en ChatGPT para la segunda pregunta:** "Desarrollar una simulación para analizar las propiedades de linealidad e invariancia en el tiempo de un sistema modelado como "caja negra". Si el sistema es lineal e invariante, determinar su respuesta al impulso y usarla para calcular la salida del sistema ante una señal de entrada específica. Además, debía comprobar y graficar los resultados."